# Convolutional Neural Networks for Computer Vision
_Example of the use of CNNs for object detection_

---

Object detection is a task in computer vision that involves identifying the presence, location, and type of one or more objects in a given photograph. It is a challenging problem that involves building upon methods for object recognition (_e.g._, where are they), object localization (_e.g._, what are their extent), and object classification (_e.g._, what are they).

In recent years, deep learning techniques are achieving state-of-the-art results for object detection, such as on standard benchmark datasets and in computer vision competitions. Notable is the “You Only Look Once”, or YOLO, a family of Convolutional Neural Networks that achieve near state-of-the-art results with a single end-to-end model that can perform object detection in real-time.

In this lab, we will first focus on the simpler problem of object localization. The localization problem considers that only one object is present on the image, while the detection problem tries to determine all the objects present on the image. Then, we will implement a simplified version of YOLO

<!-- In this tutorial, we will put into practice some of the methods discussed in class to locate objects in an image. -->

---

In [ ]:
import PIL
from PIL import Image
import os, sys

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# tf.config.list_physical_devices()
[k.device_type for k in tf.config.list_physical_devices()]

---
# PART I: Object Localization

As said, in this section, we will focus on the simpler problem of locating a single object per image, _i.e._, locating the object associated with the dominant class of a classification algorithm. 

As we have seen during the course, in both localization and detection, we try to simultaneously determine an object's position and class in the form of a bounding box. The bounding box is encoded by its width $b_w$, its height $b_h$, and its center whose coordinates are given by the pair $(b_x, b_y)$. 

<center> <img src="https://plmlab.math.cnrs.fr/wikistat/High-Dimensional-Deep-Learning/-/raw/master/VisionCNN/img/Obama.png" style="width:500;height:300px;"></center>
<caption><center><b>Figure 1:</b> Bounding box model used for localization</center></caption>

## Data exploration

To begin, download the database images. You will find all the data at the following link: [Wildlife dataset](https://drive.google.com/drive/folders/16EYaEPi4zfq96XWEtkqW7m2gVdpXORu6?usp=sharing). _For the rest of the tutorial, we will need this data to be in a `wildlife` folder, in the current folder._

If you cannot already access the dataset (_e.g._ because you are using [Google Colab](https://colab.research.google.com/)), you can download it using the following command:

In [ ]:
!git clone https://plmlab.math.cnrs.fr/chevallier-teaching/datasets/wildlife.git

The database has 4 classes, for the following 4 animals: buffalo, elephant, rhinoceros and zebra. It contains **376** images of each of the 4 classes.

<center> <img src="https://plmlab.math.cnrs.fr/chevallier-teaching/datasets/wildlife/-/raw/main/buffalo/002.jpg" width=200>
<img src="https://plmlab.math.cnrs.fr/chevallier-teaching/datasets/wildlife/-/raw/main/elephant/003.jpg" width=200>
<img src="https://plmlab.math.cnrs.fr/chevallier-teaching/datasets/wildlife/-/raw/main/rhino/003.jpg" width=200>
<img src="https://plmlab.math.cnrs.fr/chevallier-teaching/datasets/wildlife/-/raw/main/zebra/046.jpg" width=200></center>
<caption><center><b>Figure 2:</b> Examples of images from the database</center></caption>

##### <i style="color:purple">**Todo:** Display an image of the training dataset</i>

In [ ]:
### TO BE COMPLETED ### 

# Display an image of the training dataset

In [ ]:
# %load solutions/disp_img.py

##### <i style="color:purple">**Question:** What is the format used in the database to encode the labels?</i>

...

The function below will load the data and format it for classification. Some remarks about this function
* To be able to use our images in a reasonable amount of time, we will have to start by resizing them.
* We want the label to be of the form seen during the course, _i.e._, presence + bounding box + classes
* If there are several objects in the same image, we will consider only the object whose bounding box takes up the largest area in the image.




##### <i style="color:purple">**Question:** We consider square images, denoted $x$, of size $s$ and we want to store them in a `numpy`array.</i>
1. <span style="color:purple">How big should the array for the image $x$ be?</span>

Size for an image $x$:

<span style="color:teal ">[Solution]</i>

<!-- $x$ being a square image of size $s$, in color (3 channels), it is of size $(s,s,3)$. -->

2. <span style="color:purple">We also want to store the labels, denoted $y$, in a `numpy` array. What size should this array be?</span>

Size for a label $y$:

<span style="color:teal ">[Solution]</i>

<!-- $y$ is a vector that must contain "presence" + "bounding box" + "classes". It is therefore size $9 = 1+4+4$. -->

In [ ]:
from keras.utils import to_categorical

In [ ]:
### TO BE COMPLETED ###

# Total number of images
DATASET_SIZE = ...  ### TO BE COMPLETED ###

# Here we choose the dimension in which we will resize the images:
# 64x64 is a rather small size but it will allow us to have faster experiments
# 128x128 or 256x256 would give better results but at the cost of several hours of training
IMAGE_SIZE = 64

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

def load_data_localization(image_size):
    # Path to the database
    data_path = "./wildlife/"
    # Paths to the data of the 4 different classes
    paths = [data_path + "buffalo/", data_path + "elephant/", data_path + "rhino/", data_path + "zebra/"]
    # Index for adding data to the x and y variables 
    i = 0
    # Preparation of data structures for x and y
    x = np.zeros((DATASET_SIZE, ...)) ### TO BE COMPLETED ### 
    y = np.empty((DATASET_SIZE, ...)) ### TO BE COMPLETED ### 

    # Browse paths of each class
    for path in paths:

        # Browse the (sorted) files in the directory
        dirs = os.listdir(path)
        dirs.sort()

        for item in dirs:
            if os.path.isfile(path + item):
                # Extracting the file extension 
                extension =item.split(".")[1]

            if extension == "jpg" or extension == "JPG":
                # Image : we will fill the variable x
                # Reading the image
                img = Image.open(path + item)
                # Image scaling
                img = img.resize((image_size,image_size), Image.ANTIALIAS)
                # Filling the variable x
                x[i] = np.asarray(img)

            elif extension == "txt":
                # Text file: bounding box coordinates to fill y
                labels = open(path + item, "r")
                # Retrieving of lines from the text file
                labels = labels.read().split('\n')
                # If the last line is empty, delete it 
                if labels[-1] == "":
                    del labels[-1]

                # Maximum area bounding box index
                j_max = 0
                if len(labels) > 1:
                    area_max = 0 # Bounding box area of maximum area
                    # Browse bounding boxes for objects in the image
                    for j in range(len(labels)):
                        # Compute the area of the current bounding box
                        area = ... ### TO BE COMPLETED ###
                        ### Notice that labels[j].split()[k] allows access to the k-th element of the label j ###
                        # Update the maximum area bounding box, if necessary
                        if area > area_max:
                            area_max = area
                            j_max = j    

                # An object is present on the image (presence = 1)
                presence = np.array([1], dtype="i")
                # "One-hot vector " to represent the class probabilities
                classes = to_categorical(...) ### TO BE COMPLETED ###
                # Coordinates of the maximum area bounding box
                coordinates = np.array(labels[j_max].split()[1:], dtype="f")
                # Filling the variable y
                y[i, 0] = presence
                y[i, 1:5] = coordinates
                y[i, 5:] = classes

                i = i + 1
            else:
                print("extension found: ", extension)

    return x, y

In [ ]:
# %load solutions/load_data_localization.py

In [ ]:
x, y = load_data_localization(IMAGE_SIZE)

##### <i style="color:purple">**Todo:** Prepare data for training</i>

1. Separate the data into training and validation sets, place 15% of the data in the test set
2. Center-reduce the coordinates of the bounding boxes
3. Normalize the color values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
### TO BE COMPLETED ###

x_train, x_val, y_train, y_val = ...

[...]

In [ ]:
# %load solutions/data_processing.py

> From now on, we will consider $\texttt{y\_mean}$ and $\texttt{y\_std}$, respectively the mean and standard deviation of $\texttt{y\_train}$, as global variables.
>
> In particular, whenever we need to use these quantities within a function, we will be able to do so.

The following function allows to visualize the objects located on the images, _i.e._ on each image the selected bounding box, in a suitable color, and its associated class.

##### <i style="color:purple">**Todo:** Write a function to visualize the data.</i>

1. If only $\texttt{x}$ and $\texttt{y}$ are indicated, an image number is randomly drawn and the $\texttt{y}$ label associated with the image is displayed
2. If a 2nd $\texttt{y}$, named $\texttt{y\_pred}$, is indicated, then the two labels are displayed side by side, so that they can be compared
3. Finally, you can also indicate the $\texttt{id}$ of the image you wish to view.

A bounding box color is assigned to each label.

In [ ]:
### TO BE COMPLETED ###

def print_data_localization(x, y, y_pred=[], id=None, image_size=IMAGE_SIZE):
    if id==None:
        # Random drawing of an image in the database
        num_img = np.random.randint(x.shape[0]-1)
    else:
        num_img = id

    img = x[num_img]
    lab = y[num_img]

    colors = ["blue", "yellow", "red", "orange"] # Different colors for different classes
    classes = ["Buffalo", "Elephant", "Rhino", "Zebra"]

    if np.any(y_pred):
        plt.subplot(1, 2, 1)

    # Image display
    ... ### TO BE COMPLETED ###
    # Determining the class
    class_id = ... ### TO BE COMPLETED ###

    # Determining the coordinates of the bounding box in the image frame
    ax = (lab[1]*y_std[1] + y_mean[1]) * image_size
    ay = (lab[2]*y_std[2] + y_mean[2]) * image_size
    width = (lab[3]*y_std[3] + y_mean[3]) * image_size
    height = (lab[4]*y_std[4] + y_mean[4]) * image_size
    #print("x: {}, y: {}, w: {}, h:{}".format(ax,ay,width, height))
    # Determining the extrema of the bounding box, namely the minimum and maximum x/y value for each box
    p_x = [..., ...] ### TO BE COMPLETED ###
    p_y = [..., ...] ### TO BE COMPLETED ###
    # Display the bounding box in the right color
    [...] ### TO BE COMPLETED ###
    plt.title("Ground truth : Image {} - {}".format(num_img, classes[class_id]))

    if np.any(y_pred):
        plt.subplot(1, 2, 2)
        # Image display
        ... ### TO BE COMPLETED ###
        lab = y_pred[num_img]
        # Determining the class
        class_id = ... ### TO BE COMPLETED ###

        # Determining the coordinates of the bounding box in the image frame
        ax = (lab[1]*y_std[1] + y_mean[1]) * image_size
        ay = (lab[2]*y_std[2] + y_mean[2]) * image_size
        width = (lab[3]*y_std[3] + y_mean[3]) * image_size
        height = (lab[4]*y_std[4] + y_mean[4]) * image_size
        #print("x: {}, y: {}, w: {}, h:{}".format(ax,ay,width, height))
        # Determining the extrema of the bounding box
        p_x = [..., ...] ### TO BE COMPLETED ###
        p_y = [..., ...] ### TO BE COMPLETED ###
        # Display the bounding box in the right color
        [...] ### TO BE COMPLETED ###
        plt.title("Prediction: Image {} - {}".format(num_img, classes[class_id]))

In [ ]:
# %load solutions/print_data_localization.py

In [ ]:
plt.figure(figsize=(15, 100)) 

for i in range(60): #x.shape[0]):
    plt.subplot(20, 3, i+1)
    print_data_localization(x_train, y_train, image_size=IMAGE_SIZE, id=i)

plt.show()

## Useful functions

1 . Computation of the **IoU** coefficient (Intersection over Union) between real and predicted bounding boxes.

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

def compute_iou(y_true_coord, y_pred_coord):
    ### "Denormalization" of bounding box coordinates
    pred_box_xy = y_pred_coord[:, ...] * y_std[...] + y_mean[...] ### TO BE COMPLETED ###
    true_box_xy = y_true_coord[:, ...] * y_std[...] + y_mean[...] ### TO BE COMPLETED ###

    ### "Denormalization of the width and height of bounding boxes
    pred_box_wh = y_pred_coord[:, ...] * y_std[...] + y_mean[...] ### TO BE COMPLETED ###
    true_box_wh = y_true_coord[:, ...] * y_std[...] + y_mean[...] ### TO BE COMPLETED ###

    # Computation of the minimum and maximum coordinates of the real bounding box
    true_mins   = ... ### TO BE COMPLETED ###
    true_maxs   = ... ### TO BE COMPLETED ###

    # Computation of the minimum and maximum coordinates of the predicted bounding box
    pred_mins   = ... ### TO BE COMPLETED ###
    pred_maxs   = ... ### TO BE COMPLETED ###

    # Determining the intersection of bounding boxes
    intersect_mins  = tf.maximum(pred_mins, true_mins)
    intersect_maxs  = tf.minimum(pred_maxs, true_maxs)
    intersect_wh    = tf.maximum(intersect_maxs - intersect_mins, 0.)
    intersect_areas = intersect_wh[:, 0] * intersect_wh[:, 1]

    # Area of predicted and actual bounding boxes
    true_areas = ... ### TO BE COMPLETED ###
    pred_areas = ... ### TO BE COMPLETED ###

    # Area of the union of predicted and real boxes
    union_areas = ... ### TO BE COMPLETED ###

    iou_scores  = tf.truediv(intersect_areas, union_areas)
    return iou_scores

In [ ]:
# %load solutions/compute_iou.py

In [ ]:
def iou():
    def iou_metrics(y_true, y_pred):
        return compute_iou(y_true, y_pred)
    iou_metrics.__name__= "IoU"
    return iou_metrics

2. Visualization of learning quality

In [ ]:
def plot_training_analysis(history, metric='loss'):    

    loss = history.history[metric]
    val_loss = history.history['val_' + metric]

    epochs = range(len(loss))

    plt.plot(epochs, loss, 'b', linestyle="--",label='Training ' + metric)
    plt.plot(epochs, val_loss, 'g', label='Validation ' + metric)
    plt.title('Training and validation ' + metric)
    plt.legend()

## A first example of CNN for object localization

This part aims to develop a first example of a convolutional network for object localization.

To do this, complete the codes provided to obtain a localization algorithm. You can use any convolutional base of your choice, but you must pay special attention to the output layer. Actually, you will produce three different outputs:
* one characterizing the presence of an object, 
* another providing the coordinates of the bounding box, and finally, 
* a last one performing the classification.

<center> <img src="https://plmlab.math.cnrs.fr/wikistat/High-Dimensional-Deep-Learning/-/raw/master/VisionCNN/img/CNN.png" width=500></center>
<caption><center><b>Figure 3:</b> Illustration of the network architecture to be built.</center></caption>

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from tensorflow.keras.models import Model, Sequential

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

def create_model_localization(input_shape=(64, 64, 3)):

    input_layer = Input(shape=input_shape)

    ### TO BE COMPLETED ###
    conv1 = Conv2D(...)(input_layer)
    # Convolutions, Pooling, Dense,...  Up to you to form your network!
    # Your last layer must update a variable x, reused in the output layers below
    x = ...

    output_p     = Dense(..., activation=..., name='p')(x)       # Output characterizing the presence of an object
    output_coord = Dense(..., activation=..., name='coord')(x)   # Output characterizing bounding box coordinates
    output_class = Dense(..., activation=..., name='classes')(x) # Output characterizing the class probabilities

    output = [output_p, output_coord, output_class]
    model  = Model(input_layer, output)

    return model

In [ ]:
# %load solutions/create_model_localization.py

You must associate a cost function with each network output to train your network. The total cost function will be the sum of the three previously defined cost functions, weighted by weights defined in the variable `loss_weights`.

**Take the time to test different values of** `loss_weights` **depending on the evolution of the metrics you observe during the training**.

In [ ]:
from tensorflow.keras.optimizers import Adam

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

epochs     = 30
batch_size = 15
model = create_model_localization()
opt   = Adam(learning_rate=3e-4)

### TO BE COMPLETED ###
# Here put, in order, the cost functions associated with each output
loss = [..., ..., ...]

# We will associate a metric to each output: the accuracy for the two classifications,
# and the IoU defined earlier for the quality of the bounding boxes.
metrics = ['accuracy', iou(), 'accuracy']
loss_weights = [1, 1, 1] ### Vary these WEIGHTS to find an ad hoc COMBINATION ###


model.compile(loss = loss,
              optimizer = opt,
              metrics = metrics,
              loss_weights = loss_weights)

history = model.fit(x_train, [y_train[:,0], y_train[:,1:5], y_train[:,5:9]],
              epochs = epochs,
              batch_size = batch_size,
              validation_data = (x_val, [y_val[:,0], y_val[:,1:5], y_val[:,5:9]]))

In [ ]:
# %load solutions/train_model_localization.py

### Graphical validation

1. _Analysis of the results:_ Curves of the evolution of the loss function and of the IoU of the bounding boxes, as well as of the accuracy of the predicted classes

In [ ]:
def plot_results(history):
    plt.figure(figsize=(15,10))
    
    plt.subplot(2,3,1) ; plot_training_analysis(history, metric='p_loss')
    plt.subplot(2,3,2) ; plot_training_analysis(history, metric='coord_loss')
    plt.subplot(2,3,3) ;plot_training_analysis(history, metric='classes_loss')
    
    plt.subplot(2,3,4) ; plot_training_analysis(history, metric='p_accuracy') 
    plt.subplot(2,3,5) ; plot_training_analysis(history, metric='coord_io_u')
    plt.subplot(2,3,6) ; plot_training_analysis(history, metric='classes_accuracy')
    
    plt.show()

In [ ]:
plot_results(history)

2. Prediction of validation data

In [ ]:
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_val)
y_pred = np.zeros(y_val.shape)

for i in range(y_pred.shape[0]):
    y_pred[i, 0] = y_pred_presence[i][0]
    y_pred[i, 1:5] = y_pred_coords[i]
    y_pred[i, 5:9] = y_pred_classes[i]

3. Display of results on several images

In [ ]:
for i in [2,3,7,25,16,18,24,15]:
    plt.figure(figsize=(10,5))
    print_data_localization(x_val, y_val, y_pred = y_pred, id=i, image_size=IMAGE_SIZE)

plt.show()

## Improvement of the cost function

In practice, it is tricky to find a good combination of loss functions as you did on the previous cells. The cross-entropy and the mean square error give values that are too different from being combined effectively.

A variant, perhaps simpler to do work, is to use only the mean square error as the loss for all outputs. This variant is implemented in the YOLO algorithm, of which we will implement a variant in the second part of this tutorial. Test this solution below. As in the previous exercise, feel free to vary the weights of the different elements of the cost function.

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

epochs = 30
batch_size = 15
model = create_model_localization()
opt = Adam(learning_rate=3e-4)

loss = [..., ..., ...]
metrics = ['accuracy', iou(), 'accuracy']
loss_weights = [...]


model.compile(loss = loss,
              optimizer = opt,
              metrics = metrics,
              loss_weights = loss_weights)

history = model.fit(x_train, [y_train[:,0], y_train[:,1:5], y_train[:,5:9]],
              epochs = epochs,
              batch_size = batch_size,
              validation_data = (x_val, [y_val[:,0], y_val[:,1:5], y_val[:,5:9]]))

In [ ]:
# %load solutions/train_model_localization_MSE.py

### Graphical validation

1. _Analysis of the results:_ Curves of the evolution of the loss function and of the IoU of the bounding boxes, as well as of the accuracy of the predicted classes

In [ ]:
plot_results(history)

2. Prediction of validation data

In [ ]:
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_val)
y_pred = np.zeros(y_val.shape)

for i in range(y_pred.shape[0]):
    y_pred[i, 0] = y_pred_presence[i][0]
    y_pred[i, 1:5] = y_pred_coords[i]
    y_pred[i, 5:9] = y_pred_classes[i]

3. Display of results on several images

In [ ]:
for i in [2,3,7,15,16,18,24,25]:
    plt.figure(figsize=(10,5))
    print_data_localization(x_val, y_val, y_pred = y_pred, id=i, image_size=IMAGE_SIZE)

plt.show()

### Some axes of improvement

Considering the small size of the database, the results are not bad! There is some confusion between some classes, but the predictions are often interesting.

However, there should still be strong overfitting at this stage. There are several possibilities to correct it: 
 
* _Regularization_ by weight decay, using `kernel_regularizer` on the network layers
* _Data augmentation_: For example, with a [`PyDataset`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/PyDataset) class and the use of the [`Albumentation`](https://albumentations.ai/) library.
* Use of _transfer learning_: Starting from a network trained on *ImageNet* (which contains many classes of animals), we would benefit from very general filters which would help to limit overfitting. 

#### Data augmentation

In [ ]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

In [ ]:
import albumentations as A
import cv2

In [ ]:
AUGMENTATIONS_TRAIN = A.Compose([
    #A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


AUGMENTATIONS_TEST = A.Compose([
    A.ShiftScaleRotate(p=0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [ ]:
from tensorflow.keras.utils import PyDataset

In [ ]:
class WildLifeSequence(PyDataset):
    # Initialization of the sequence with different parameters
    def __init__(self, x_set, y_set, batch_size,augmentations):
        self.x, self.y = x_set, y_set
        self.classes = ["Buffalo", "Elephant", "Rhino", "Zebra"]
        self.batch_size = batch_size
        self.augment = augmentations
        self.indices1 = np.arange(x_set.shape[0])
        np.random.shuffle(self.indices1)
        # The indices to access the data are randomized at each epoch to vary the composition of the batches during the training


    # Computation of the number of gradient descent steps per epoch
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))


    # Rounding problems in bounding box conversions internal to the Albumentations library
    # To get around them, if the boxes are too close to the edges, we erode them a little
    def erode_bounding_box(self, box):
        epsilon = 0.01

        xmin = max(box[0] - box[2]/2, epsilon)
        ymin = max(box[1] - box[3]/2, epsilon)
        xmax = min(box[0] + box[2]/2, 1-epsilon)
        ymax = min(box[1] + box[3]/2, 1-epsilon)

        cx = xmin + (xmax - xmin)/2
        cy = ymin + (ymax - ymin)/2
        width = xmax - xmin
        height = ymax - ymin

        return np.array([cx, cy, width, height])


    # Apply data augmentation to each image in the batch and associated bounding boxes
    def apply_augmentation(self, bx, by):

        batch_x = np.zeros((bx.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
        batch_y = by

        # For each image in the batch
        for i in range(len(bx)):
            bboxes = []
            box = by[i,1:5]
            # Denormalization of bounding box coordinates
            box[0] = (box[0]*y_std[1] + y_mean[1])
            box[1] = (box[1]*y_std[2] + y_mean[2])
            box[2] = (box[2]*y_std[3] + y_mean[3])
            box[3] = (box[3]*y_std[4] + y_mean[4])
            box = self.erode_bounding_box(box)
            bboxes.append(box)

            class_labels = []
            class_id = np.argmax(by[i, 5:])
            class_labels.append(self.classes[class_id])

            img = bx[i]

            # Application of augmentation to images and masks
            transformed = self.augment(image=img.astype('float32'), 
                                       bboxes=bboxes, class_labels=class_labels)
            batch_x[i] = transformed['image']
            batch_y_transformed = transformed['bboxes']

            # Renormalization of transformed bounding box coordinates
            batch_y[i, 1] = (batch_y_transformed[0][0] - y_mean[1])/y_std[1]
            batch_y[i, 2] = (batch_y_transformed[0][1] - y_mean[2])/y_std[2]
            batch_y[i, 3] = (batch_y_transformed[0][2] - y_mean[3])/y_std[3]
            batch_y[i, 4] = (batch_y_transformed[0][3] - y_mean[4])/y_std[4]

        return batch_x, batch_y


    # Function called at each new batch: selection and increase of data
    def __getitem__(self, idx):
        batch_x = self.x[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_y = self.y[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_x, batch_y = self.apply_augmentation(batch_x, batch_y)

        batch_x = np.array(batch_x, dtype=np.float32)
        batch_y = np.array(batch_y, dtype=np.float32)
        batch_y = batch_y[:,0], batch_y[:,1:5], batch_y[:,5:9]
        return batch_x, batch_y

    # Function called at the end of an epoch: randomization of data access indices.
    def on_epoch_end(self):
        np.random.shuffle(self.indices1)

In [ ]:
# Instantiation of a Sequence
batch_size = 16

train_gen = WildLifeSequence(x_train, y_train, batch_size, augmentations=AUGMENTATIONS_TRAIN)

# To test the sequence, we select the elements of the first batch and display them
batch_x, batch_y = train_gen.__getitem__(0)

y_batch = np.zeros((batch_y[0].shape[0],9))

for i in range(batch_y[0].shape[0]):
    y_batch[i, 0] = batch_y[0][i]
    y_batch[i, 1:5] = batch_y[1][i]
    y_batch[i, 5:9] = batch_y[2][i]

# --- #

plt.figure(figsize=(15, 25))

for i in range(15): 
    plt.subplot(5, 3, i+1)
    print_data_localization(batch_x, y_batch, image_size=IMAGE_SIZE, id=i)

plt.show()

In [ ]:
model = create_model_localization()
opt = Adam(learning_rate=3e-4)
batch_size = 18
epochs = 50

loss=['binary_crossentropy', 'mse', 'categorical_crossentropy']
metrics=['accuracy',iou(),'accuracy']
loss_weights = [1, 5, 1]

train_gen = WildLifeSequence(x_train, y_train, batch_size, augmentations=AUGMENTATIONS_TRAIN)
valid_gen = WildLifeSequence(x_val, y_val, batch_size, augmentations=AUGMENTATIONS_TEST)

model.compile(loss = loss,
              optimizer = opt,
              metrics = metrics,
              loss_weights = loss_weights)
#model.summary()

history = model.fit(train_gen,
                    epochs = epochs,
                    validation_data = valid_gen)

1. _Analysis of the results:_ Curves of the evolution of the loss function and of the IoU of the bounding boxes, as well as of the accuracy of the predicted classes

In [ ]:
plot_results(history)

2. Prediction of validation data

In [ ]:
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_val)
y_pred = np.zeros(y_val.shape)

for i in range(y_pred.shape[0]):
    y_pred[i, 0] = y_pred_presence[i][0]
    y_pred[i, 1:5] = y_pred_coords[i]
    y_pred[i, 5:9] = y_pred_classes[i]

3. Display of results on several images

In [ ]:
for i in [2,3,7,25,16,18,24,15]:
    plt.figure(figsize=(10,5))
    print_data_localization(x_val, y_val, y_pred = y_pred, id=i, image_size=IMAGE_SIZE)

plt.show()

#### Transfer learning

In [ ]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False, # The Dense part of the original network is not kept
                  input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

##### <i style="color:purple">**Todo:** Write a function to create a neural network based on the $\texttt{VGG-16}$ convolution basis.</i>

In [ ]:
### TO BE COMPLETED ###

def create_model_localization(input_shape=(64, 64, 3)):
    [...]

In [ ]:
# %load solutions/create_model_localization_VGG.py

In [ ]:
model = create_model_localization_VGG()
model.summary()

##### <i style="color:purple">**Todo:** Achieve Transfer Learning.</i>

In [ ]:
### TO BE COMPLETED ###

[...]

In [ ]:
#%load solutions/transfert_learning.py

1. _Analysis of the results:_ Curves of the evolution of the loss function and of the IoU of the bounding boxes, as well as of the accuracy of the predicted classes

In [ ]:
plot_results(history)

2. Prediction of validation data

In [ ]:
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_val)
y_pred = np.zeros(y_val.shape)

for i in range(y_pred.shape[0]):
    y_pred[i, 0] = y_pred_presence[i][0]
    y_pred[i, 1:5] = y_pred_coords[i]
    y_pred[i, 5:9] = y_pred_classes[i]

3. Display of results on several images

In [ ]:
for i in [2,3,7,25,16,18,24,15]:
    plt.figure(figsize=(10,5))
    print_data_localization(x_val, y_val, y_pred = y_pred, id=i, image_size=IMAGE_SIZE)

plt.show()

#### Data augmentation & Transfert learning

##### <i style="color:purple">**Todo:** Combine Transfer Learning and Data Augmentation.</i>

In [ ]:
### TO BE COMPLETED ###

[...]

In [6]:
# %load solutions/transfert_learning_data_augment.py

In [ ]:
plot_results(history)

2. Prediction on validation data

In [ ]:
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_val)
y_pred = np.zeros(y_val.shape)

for i in range(y_pred.shape[0]):
    y_pred[i, 0] = y_pred_presence[i][0]
    y_pred[i, 1:5] = y_pred_coords[i]
    y_pred[i, 5:9] = y_pred_classes[i]

3. Results display on several images

In [ ]:
for i in [2,3,7,25,16,18,24,15]:
    plt.figure(figsize=(10,5))
    print_data_localization(x_val, y_val, y_pred = y_pred, id=i, image_size=IMAGE_SIZE)

plt.show()

---
# PART II: Implementation of a simplified version of YOLO

In this part, we will try to go further by considering the more complex problem of object detection, _i.e._, the joint localization and classification of all the objects in the image; for that, we will implement a simplified version of YOLO.

The simplification comes from the fact that we will not include all the elements described in [Redmon](https://pjreddie.com/)'s article (for example, the optimizer's choice). One of the main simplifications is that _we will only consider one object per cell_.

<center> <img src="https://plmlab.math.cnrs.fr/wikistat/High-Dimensional-Deep-Learning/-/raw/master/VisionCNN/img/YOLO_archi.png" width=500></center>
<caption><center><b>Figure 4</b>: Pipeline of the YOLO algorithm <a href="https://pjreddie.com/media/files/papers/yolo_1.pdf">[Redmon 2016]</a></center></caption>

As a reminder, the idea of YOLO is to slice the image into a grid of cells and perform a prediction of several bounding boxes as well as a classification per cell.

We use here the same dataset : the [Wildlife](https://drive.google.com/drive/folders/16EYaEPi4zfq96XWEtkqW7m2gVdpXORu6?usp=sharing) database.

## Useful functions

Definition of the different variables useful for the following.

In [ ]:
IMAGE_SIZE = 64  # Size of the input images of the network
CELL_PER_DIM = 8 # Number of cells in width and height
BOX_PER_CELL = 1 # Number of objects per cell
NB_CLASSES = 4   # Number of classes of the problem
PIX_PER_CELL = IMAGE_SIZE/CELL_PER_DIM

DATASET_SIZE = 376*4

##### <i style="color:purple">**Question:** What is the size of the $y$-array in the YOLO framework?</i>

Size for a label $y$:

<span style="color:teal ">[Solution]</i>

<!-- $y$ is a vector of size $(S,S,N+5k)$, where:
* $S$ is the grid size, or with the notations introduced above `CELL_PER_DIM`,
* $N$ is the number of classes, _i.e._, `NB_CLASSES`, and
* $k$ is the number of objects detected per cell _i.e._ `BOX_PER_CELL`.

In other words, $y$ is of size (`CELL_PER_DIM`,`CELL_PER_DIM`,`NB_CLASSES`+5*`BOX_PER_CELL`). -->

### Data loading and formatting for the detection problem. 

The function defined hereafter is the pendant of the `load_data_detection` function previously defined.

Problem data (more than one bounding box per cell) are indicated and displayed during loading.

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

def load_data_detection():
    # Path to the database
    data_path = "./wildlife/"
    # Paths to the data of the 4 different classes
    paths = [data_path+"buffalo/", data_path+"elephant/", data_path+"rhino/", data_path+"zebra/"]
    # Index for adding data to the x and y variables
    i = 0
    # Preparation of data structures for x and y
    x = np.zeros((DATASET_SIZE, ...)) ### TO BE COMPLETED ###
    y = np.zeros((DATASET_SIZE, ...)) ### TO BE COMPLETED ###

    # Save normalized bounding box width/height
    widths = []
    heights = []

    # Browse paths of each class
    for path in paths:

        # Browse the (sorted) files in the directory
        dirs = os.listdir(path)
        dirs.sort()

        for item in dirs:
            if os.path.isfile(path + item):
                # Extracting the file extension
                extension = item.split(".")[1]

                if extension=="jpg" or extension=="JPG":
                    # Image : we will fill the variable x
                    # Reading the image
                    img = PIL.Image.open(path + item)
                    # Image scaling
                    img = img.resize((image_size,image_size), PIL.Image.Resampling.LANCZOS)
                    # Filling the variable x
                    x[i] = np.asarray(img, dtype=np.int32)

                elif extension=="txt":
                    # Text file: bounding box coordinates to fill y
                    labels = open(path + item, "r")
                    # Retrieving of lines from the text file
                    labels = labels.read().split('\n')
                    # If the last line is empty, delete it
                    if labels[-1]=="":
                        del labels[-1]

                    err_flag = 0
                    boxes = []
                    for label in labels:
                        # Retrieving information from the bounding box
                        label = label.split()
                        # Save width/height of bounding boxes
                        widths.append(float(...))  ### TO BE COMPLETED ###
                        heights.append(float(...)) ### TO BE COMPLETED ###
                        # Bounding box center coordinates in the image frame
                        cx, cy = float(...) * IMAGE_SIZE, float(...) * IMAGE_SIZE ### TO BE COMPLETED (x2) ###
                        # Determination of the indices of the cell in which the center falls
                        ind_x, ind_y = int(cx // PIX_PER_CELL), int(cy // PIX_PER_CELL)
                        # YOLO : "The (x, y) coordinates represent the center of the box relative to the bounds of the grid cell."
                        # -> Compute the coordinates of the center relative to the cell in which it is located
                        cx_cell = (cx - ind_x * PIX_PER_CELL) / PIX_PER_CELL
                        cy_cell = (cy - ind_y * PIX_PER_CELL) / PIX_PER_CELL
                        # Confidence index of the bounding box
                        presence = np.array(...) ### TO BE COMPLETED ###
                        # "One-hot vector " to represent the class probabilities
                        classes = ... ### TO BE COMPLETED ###
                        # We arrange the class probabilities at the end of the vector ([ BOX 1 ; BOX 2 ; ... ; BOX N ; CLASSES])
                        y[i, ind_x, ind_y, 5 * BOX_PER_CELL:] = classes

                        boxes.append([cx, cy, label[3]*IMAGE_SIZE, label[4]*IMAGE_SIZE])
                        # Determining the cell bounding box index in which to store the information
                        ind_box = 0
                        while y[i, ind_x, ind_y, 5*ind_box] == 1 and ind_box < BOX_PER_CELL - 1:
                            # If the current index box is already in use (presence = 1)
                            # and the maximum number of boxes has not been reached, we go to the next box
                            ind_box = ind_box + 1

                        if y[i, ind_x, ind_y, 5*ind_box] == 1:
                            print("ERROR: THE CELL ALREADY CONTAINS ALL THE AVAILABLE BOXES")
                            print(path + item)
                            err_flag = 1
                        else:
                            y[i, ind_x, ind_y, 5*ind_box] = 1
                            y[i, ind_x, ind_y, 5*ind_box + 1] = cx_cell
                            y[i, ind_x, ind_y, 5*ind_box + 2] = cy_cell
                            # Square root of the width and height of the box
                            y[i, ind_x, ind_y, 5*ind_box + 3] = math.sqrt(float(...)) ### TO BE COMPLETED : width ###
                            y[i, ind_x, ind_y, 5*ind_box + 4] = math.sqrt(float(...)) ### TO BE COMPLETED : height ###

                    i = i + 1
                    if err_flag == 1:
                        img_name = item.split(".")[0]
                        img = Image.open(path + img_name + '.jpg')
                        # Image scaling
                        img = PIL.img.resize((IMAGE_SIZE, IMAGE_SIZE), PIL.Image.Resampling.LANCZOS)

                        plt.imshow(img)
                        for ind_cell in range(CELL_PER_DIM):
                            plt.plot([ind_cell*PIX_PER_CELL, ind_cell*PIX_PER_CELL], [0, IMAGE_SIZE-1], 'k-')
                            plt.plot([0, IMAGE_SIZE-1], [ind_cell*PIX_PER_CELL, ind_cell*PIX_PER_CELL], 'k-')

                        for ind_box_plot in range(len(boxes)):
                            box = boxes[ind_box_plot]
                            plt.plot(box[0], box[1], 'b.')
                        plt.show()

                else:
                    print("extension found: ", extension)

    return x, y, widths, heights

In [ ]:
# %load solutions/load_data_detection.py

In [ ]:
x, y, w, h = load_data_detection()

As before, the database is divided between training and validation data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, random_state=42)

# Image normalization
x_train = x_train/255
x_val = x_val/255

x_train.shape, y_train.shape, x_val.shape, y_val.shape

### Display of data and detection results

In [ ]:
from scipy.special import softmax

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

def print_data_detection(x, y, id=None, image_size=IMAGE_SIZE, mode='gt'):
    if id==None:
        # Random drawing of an image in the database
        num_img = np.random.randint(x.shape[0]) 
        print(num_img)
    else:
        num_img = id

    img = x[num_img]
    lab = y[num_img]

    colors = ["blue", "yellow", "red", "orange"] # Different colors for the different classes
    classes = ["Buffalo", "Elephant", "Rhino", "Zebra"]

    boxes = lab[:, :, 1:5]
    for ind_x in range(CELL_PER_DIM):
        for ind_y in range(CELL_PER_DIM):
            box = boxes[ind_x, ind_y]
            box[0] = ... ### TO BE COMPLETED ### 
            box[1] = ... ### TO BE COMPLETED ### 
            box[2] = box[2]**2 * IMAGE_SIZE
            box[3] = box[3]**2 * IMAGE_SIZE
            boxes[ind_x, ind_y] = box

    # Retrieve all information from bounding boxes
    all_presences = np.reshape(lab[:, :, 0], (CELL_PER_DIM*CELL_PER_DIM))
    all_boxes = np.reshape(lab[:, :, 1:5], (-1, 4))
    all_classes = np.reshape(lab[:, :, 5:9], (-1, 4))

    if mode=='pred':
        all_presences = 1 / (1 + np.exp(-all_presences))
        all_classes = softmax(all_classes, axis=1)

    indices_sorted = np.argsort(-all_presences)

    # Eliminate all bounding boxes whose probability of presence is < threshold 
    threshold = 0.35
    all_boxes = ... ### TO BE COMPLETED ### 
    all_classes = ... ### TO BE COMPLETED ### 
    all_presences = ... ### TO BE COMPLETED ### 


    # Image display
    plt.imshow(img)
    for i in range(all_boxes.shape[0]):

        # Determination of the class
        class_id = ... ### TO BE COMPLETED ###
        lab = all_boxes[i]
        # Determination of the extrema of the bounding box
        p_x = [..., ...] ### TO BE COMPLETED ###
        p_y = [..., ...] ### TO BE COMPLETED ###
        # Display the bounding box in the right color
        [...] ### TO BE COMPLETED ###
  
    plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.show()  

In [ ]:
# %load solutions/print_data_detection.py

In [ ]:
print_data_detection(x_train, y_train, image_size=IMAGE_SIZE)

## A simplified version of YOLO

The model proposed below is only one possibility among many others. The Redmon article mentions a delicate instability during the training. So, we chose to use an elu (exponential linear unit) activation function.

<center> <img src="https://plmlab.math.cnrs.fr/wikistat/High-Dimensional-Deep-Learning/-/raw/master/VisionCNN/img/YOLO.png" width=500></center>
<caption><center><b>Figure 5</b>: YOLO output layer</a></center></caption>

In [ ]:
from tensorflow.keras.layers import Reshape
from tensorflow.keras import regularizers

##### <i style="color:purple">**Todo:** Complete the last layer to have the right size output.</i>



In [ ]:
### TO BE COMPLETED ###

def create_model_YOLO(input_shape=(64, 64, 3)):
    weight_decay = 0

    input_layer = Input(shape=input_shape)

    conv1 = Conv2D(32, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(input_layer)
    conv1 = Conv2D(32, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = Conv2D(32, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(64, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = Conv2D(64, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = Conv2D(128, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    dense4 = Flatten()(pool3)
    dense4 = Dense(512, activation='elu',kernel_regularizer=regularizers.l2(weight_decay))(dense4)
    dense5 = Dense(512, activation='elu',kernel_regularizer=regularizers.l2(weight_decay))(dense4)
    output = Dense(..., activation='linear',kernel_regularizer=regularizers.l2(weight_decay))(dense5) ### TO BE COMPLETED ###
    output = Reshape((...))(output) ### TO BE COMPLETED ###

    model = Model(input_layer, output)

    return model

In [ ]:
# %load solutions/create_model_YOLO.py

In [ ]:
model = create_model_YOLO()
model.summary()

### Cost function

We now come to the tricky part of the YOLO implementation: the definition of the cost function to use.

<center> <img src="https://plmlab.math.cnrs.fr/wikistat/High-Dimensional-Deep-Learning/-/raw/master/VisionCNN/img/YOLOss.png" style="width:500;height:300px;"></center>
<caption><center> Detail of the loss function defined in the YOLO v1 article</center></caption>

When writing a custom cost function in `keras`, it is necessary to use only the functions on the following page: https://keras.rstudio.com/articles/backend.html. 
Indeed this cost function, which will be called during the training, will treat tensors, not `numpy` arrays. We must therefore use the `tensorflow` library, which allows us to manipulate tensors.

An essential part of the function is already written: the one that allows the separation of the data of the so-called "empty" cells (the ground truth does not contain a bounding box) from the "non-empty" ones.

The details of the cost function are shown above: in the article $\lambda_{\text{coord}} = 5$ and $\lambda_{\text{noobj}} = 0.5$. 
* The $x_i$, $y_i$, $w_i$, and $h_i$ correspond to the coordinates of a bounding box; 
* $C_i$ corresponds to the probability of the presence of an object in the cell (sigmoid function applied to the corresponding output elements); and 
* the $p_i(c)$ are the class probabilities (softmax function applied to the corresponding output elements).

In [ ]:
from keras import backend as K
from keras.layers import Input

##### <i style="color:purple">**Todo:** Complete the following function</i>

In [ ]:
### TO BE COMPLETED ###

# Definition of the YOLO loss function
def YOLOss(lambda_coord, lambda_noobj, batch_size):

    # "Green" part: subpart concerning the confidence index
    # and the class probabilities probabilities in the case where a box is present in the cell
    def box_loss(y_true, y_pred):
        return ... ### TO BE COMPLETED ###

    # "Blue" part: subpart concerning the coordinates of the bounding box in the case where a box is present in the cell
    def coord_loss(y_true, y_pred):
        return ... ### TO BE COMPLETED ###


    # "Red" part: subpart concerning the confidence index in case no box is present in the cell
    def nobox_loss(y_true, y_pred):
        return ... ### TO BE COMPLETED ###


    def YOLO_loss(y_true, y_pred):

        # Reshape the tensors from bs x S x S x (5B+C) to (bsxSxS) x (5B+C)
        y_true = tf.reshape(y_true, shape=(-1, 9))
        y_pred = tf.reshape(y_pred, shape=(-1, 9))

        # Search (in y_true labels) for indices of cells for which at least the first bounding box is present
        not_empty = tf.greater_equal(y_true[:, 0], 1)
        indices = tf.range(0, tf.shape(y_true)[0], delta=1)
        indices_notempty_cells = indices[not_empty]

        empty = tf.less_equal(y_true[:, 0], 0)
        indices_empty_cells = indices[empty]

        # Separate the cells of y_true and y_pred with or without bounding box
        y_true_notempty = tf.gather(y_true, indices_notempty_cells)
        y_pred_notempty = tf.gather(y_pred, indices_notempty_cells)

        y_true_empty = tf.gather(y_true, indices_empty_cells)
        y_pred_empty = tf.gather(y_pred, indices_empty_cells)

        return (box_loss(y_true_notempty, y_pred_notempty) + lambda_coord*coord_loss(y_true_notempty, y_pred_notempty) + lambda_noobj*nobox_loss(y_true_empty, y_pred_empty))/batch_size


    # Return a function
    return YOLO_loss

In [ ]:
# %load solutions/YOLOss.py

### Training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
model = create_model_YOLO()
opt = Adam(learning_rate=1e-4)
batch_size = 18
epochs = 100

# As the training is unstable, we trigger a model backup each time the validation loss reaches a new minimum
model_saver = ModelCheckpoint('yolo.weights.h5', monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')


loss=[YOLOss(5, 0.5, batch_size)]

model.compile(loss = loss,
              optimizer = opt)

history = model.fit(x_train, y_train,
              epochs = epochs,
              batch_size = batch_size,
              validation_data = (x_val, y_val),
              callbacks = [model_saver])

1. Test of the version at the end of the training

In [ ]:
y_pred = model.predict(x_train)

for i in [2,3,8,15,18,24,26,32]:
    plt.figure(figsize=(5,5))
    print_data_detection(x_train, y_pred, id=i, image_size=IMAGE_SIZE, mode='pred')

plt.show()

In [ ]:
y_pred = model.predict(x_val)

for i in [2,3,8,15,18,24,26,32]:
    plt.figure(figsize=(5,5))
    print_data_detection(x_val, y_pred, id=i, image_size=IMAGE_SIZE, mode='pred')

plt.show()

2. Test the _best_ saved version

In [ ]:
model.load_weights('yolo.weights.h5')

In [ ]:
y_pred = model.predict(x_train)

for i in [2,3,8,15,18,24,26,32]:
    plt.figure(figsize=(5,5))
    print_data_detection(x_train, y_pred, id=i, image_size=IMAGE_SIZE, mode='pred')

plt.show()

In [ ]:
y_pred = model.predict(x_val)

for i in [2,3,8,15,18,24,26,32]:
    plt.figure(figsize=(5,5))
    print_data_detection(x_val, y_pred, id=i, image_size=IMAGE_SIZE, mode='pred')

plt.show()

### Loading weights from an already trained network

As YOLO training is very unstable, it is possible that at the end of this tutorial you will not get very convincing results. To finish this tutorial, you will find below the weights of a model trained for a long time.

In [ ]:
!wget --no-check-certificate 'https://plmbox.math.cnrs.fr/f/a22217b682014195b580/?dl=1' -O best_weights.index
!wget --no-check-certificate 'https://plmbox.math.cnrs.fr/f/39e2f71eda1348618c03/?dl=1' -O best_weights.data-00000-of-00001

As this model has been trained with an older version of $\texttt{Adam}$, we need to (slightly) adapt the model definition.

In [ ]:
!pip install tf-keras 

In [ ]:
TF_USE_LEGACY_KERAS = 1
from tensorflow.keras.optimizers import Adam

In [ ]:
model = create_model_YOLO()
opt = Adam(learning_rate=1e-4)

model.compile(optimizer=opt)
model.load_weights('best_weights')

In [ ]:
y_pred = model.predict(x_train)

for i in [2,3,8,15,18,24,26,32]:
    plt.figure(figsize=(5,5))
    print_data_detection(x_train, y_pred, id=i, image_size=IMAGE_SIZE, mode='pred')

plt.show()

In [ ]:
y_pred = model.predict(x_val)

for i in [2,3,8,15,18,24,26,32]:
    plt.figure(figsize=(5,5))
    print_data_detection(x_val, y_pred, id=i, image_size=IMAGE_SIZE, mode='pred')

plt.show()

The results are not perfect, but we are starting to see some good results. As previously, we could limit  overfitting by using data augmentation in this training. 

We can already see in the few examples below that some of the images are rather well-predicted.

In [ ]:
y_pred = model.predict(x_val)

for i in [28,37,39,81,108,193,214,220]:
    plt.figure(figsize=(5,5))
    print_data_detection(x_val, y_pred, id=i, image_size=IMAGE_SIZE, mode='pred')

plt.show()